In [3]:
import pandas as pd

raw_producao = pd.read_excel('Teste – Estágio de Processos - rev01 (20241709).xlsx', sheet_name='Produção')
raw_paradas = pd.read_excel('Teste – Estágio de Processos - rev01 (20241709).xlsx', sheet_name='Paradas')

raw_producao.columns = ['Ano', 'Mes', 'Dia', 'Data', 'Hora Inicio', 'Hora Fim', 'Codigo do Item', 'Ordem de Producao',
                       'Horas Totais', 'Horas Paradas', 'Horas Efetivas', 'kg', 't/h', 'Horas OE', 'Horas Disponiveis', 'Turno']

raw_paradas.columns = ['Ano', 'Mes', 'Dia', 'Data', 'Codigo do Item', 'Ordem de Producao', 'Hora Inicio', 'Hora Fim', 'ID', 'Parada',
                      'Tempo de Parada (decimal)', 'Tempo de Parada (hh:mm)', 'Turno', 'Equipamento', 'Fato', 'OE', 'Tipo de Parada']

raw_producao['Data'] = pd.to_datetime(raw_producao['Data'])
raw_paradas['Data'] = pd.to_datetime(raw_paradas['Data'])

df_producao = raw_producao.copy()
df_paradas = raw_paradas.copy()

#Excluindo tempos de parada inferior a 4 minutos
df_paradas['Tempo de Parada (hh:mm)'] = df_paradas['Tempo de Parada (hh:mm)'].astype(str)
df_paradas['Tempo de Parada (hh:mm)'] = pd.to_timedelta(df_paradas['Tempo de Parada (hh:mm)'])
df_paradas = df_paradas[df_paradas['Tempo de Parada (hh:mm)'] >= pd.Timedelta(minutes=4)]

# Excluindo todos t/h maiores que 45 toneladas por hora
df_producao = df_producao[df_producao['t/h'] < 45.1]

In [4]:
# Filtrar paradas OE
paradas_oe = df_paradas[df_paradas['OE'] == 'Sim']

tempo_total_paradas_oe = paradas_oe['Tempo de Parada (decimal)'].sum()

print(f"Tempo total de paradas OE: {tempo_total_paradas_oe:.2f} horas")

Tempo total de paradas OE: 1034.03 horas


In [5]:
# Calculando a eficiencia produtiva (horas efetivas / horas disponiveis)
df_producao['Eficiencia Produtiva (%)'] = (df_producao['Horas Efetivas'] / df_producao['Horas Disponiveis']) * 100

media_eficiencia = df_producao['Eficiencia Produtiva (%)'].mean()

print(f"Média da eficiencia produtiva: {media_eficiencia:.2f}%")

Média da eficiencia produtiva: 77.62%


In [6]:
# Contar as causas mais comuns de paradas OE
principais_paradas_oe = paradas_oe['Parada'].value_counts().head(5)

print("Principais causas de paradas OE: \n")
print(principais_paradas_oe)

Principais causas de paradas OE: 

Parada
Silo de Ensaque Cheio            432
Aguardando liberar silo          393
Manutenção Mecânica Corretiva    359
Produto de baixo rendimento      231
Limpeza Programada               192
Name: count, dtype: int64


In [10]:
# Analisando os equipamentos mais criticos
equipamentos_criticos = paradas_oe['Equipamento'].value_counts().head(3)

print("Equipamentos mais criticos com paradas OE:\n")
print(equipamentos_criticos)

# Sugestoes de melhorias baseadas nos equipamentos e causas de paradas mais frequentes
sugestoes = """
Sugestões de Melhorias:
1. Aumentar a manutencao preventiva nos equipamentos mais criticos, como CCM001 e TR0102.
2. Implementar monitoramento em tempo real para detectar falhas antes que ocorram paradas OE prolongadas.
3. De acordo com as principais causas de parada:
    I.Silo de Ensaque Cheio:
        Instalar sensores que avisem quando o silo esta quase cheio.
    II.Aguardando Liberar Silo:
        Definir quem eh responsavel pela liberacao e automatizar um pouco esse processo.
    III.Manutenção Mecânica Corretiva:
        Fazer manutencao preventiva, mas achar modos de prever quando for necessaria.
    IV.Produto de Baixo Rendimento:
        Rever a qualidade dos insumos
    V.Limpzea Programada:
        Ajustar o cronograma para momentos de menor movimento.
    
"""


Equipamentos mais criticos com paradas OE:

Equipamento
MM-0301/MM-0302    105
SI-0202             72
EN0506              53
Name: count, dtype: int64


In [11]:
# Sugestoes
print(sugestoes)


Sugestões de Melhorias:
De acordo com as principais causas de parada:
    I.Silo de Ensaque Cheio:
        Instalar sensores que avisem quando o silo esta quase cheio.
    II.Aguardando Liberar Silo:
        Definir quem eh responsavel pela liberacao e automatizar um pouco esse processo.
    III.Manutenção Mecânica Corretiva:
        Fazer manutencao preventiva, mas achar modos de prever quando for necessaria.
    IV.Produto de Baixo Rendimento:
        Rever a qualidade dos insumos
    V.Limpzea Programada:
        Ajustar o cronograma para momentos de menor movimento.
    

